In [ ]:
import pandas as pd
import nltk
import os
from textblob import TextBlob

In [ ]:
#CSV file with extracted tweets for CWC19Final hash-tag during before the event
file_1 = pd.read_csv("D:\\LJMU\\Project Topics\\Data\\Cleansed Data\\output_got_beforematch.csv",error_bad_lines=False,delimiter=";")
#CSV file with extracted tweets for EngVsNZ hash-tag during before the event
file_2 = pd.read_csv("D:\\LJMU\\Project Topics\\Data\\Cleansed Data\\output_got_ENGvsNZ.csv",error_bad_lines=False, delimiter =";", quotechar='\"')
#CSV file with extracted tweets for NZVsENG hash-tag during before the event
file_3 = pd.read_csv("D:\\LJMU\\Project Topics\\Data\\Cleansed Data\\output_got_NZvsENG.csv",error_bad_lines=False,delimiter=";")


#Combine the dataframes
interim = file_1.append(file_2)
combine = interim.append(file_3)

len(combine)


In [ ]:
combine.head(10)


In [ ]:
#Remove duplicates from the dataset
combine = combine.drop_duplicates()

len(combine)

In [ ]:
# Removing redundant columns
combine = combine.drop(['hashtags'],axis=1)
combine = combine.drop(['geo'],axis=1)
combine = combine.drop(['mentions'],axis=1)
combine = combine.drop(['permalink'],axis=1)
combine = combine.drop(['permalink,,,,,,,,'],axis=1)
#combine = combine.drop(index=[7,8],axis=1)

In [ ]:
combine.head(3)

In [ ]:
#Data cleaning starts from here. Processing of text column




